In [13]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)


c:\Users\Admin\Data\multimodal-rag-baseline


In [14]:
from configs.config import *
from libs.common import *
load_dotenv(find_dotenv())

True

In [ ]:
dataset = load_dataset("hwyin04/table-cross-page")

save_dir = f"{data_dir}/pdf"
os.makedirs(save_dir, exist_ok=True)

for split in dataset.keys():
    path = os.path.join(save_dir, f"{split}.jsonl")
    dataset[split].to_json(path, orient="records", lines=True)
    print(f"Saved {split} to {path}")


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 970.01ba/s]

Saved train to c:/Users/Admin/Data/multimodal-rag-baseline\data/pdf\train.jsonl


In [4]:

def download_pdfs_from_jsonl(jsonl_file):
    with open(jsonl_file, 'r') as f:
        for line in f:
            data = json.loads(line.strip())
            
            pdf_data = data.get('pdf')
            if pdf_data and 'path' in pdf_data:
                pdf_path = pdf_data['path']

                if pdf_path:
                    try:
                        shutil.copy(pdf_path, save_dir)
                        print(f"Đã tải: {pdf_path}")
                    except Exception as e:
                        print(f"Không thể tải {pdf_path}: {e}")
            else:
                print(f"Không tìm thấy khóa 'path' trong dữ liệu: {data}")

download_pdfs_from_jsonl(f'{save_dir}/train.jsonl')


Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\0c92f65db928c431023f59603039aa1e.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\0ed7f2ecfd607a42c745b0889e500ecf.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\0f2eac764c8d04d8d13d8a999342d106.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\2c98e99a08ec5392d50e60370d871319.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\2d64938a8d3e12dbb709f760ecd53e14.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\2e1b34aa28e718f66

In [ ]:
tables_sources = json.load(open("results.json"))
sources = list(tables_sources.keys())
for source in sources:
    print(f"source {source} has {len(tables_sources[source])} tables")
    print("-" * 100)
    # Loop through each table in the source
    for table in tables_sources[source]:
        
        ## Mỗi 1 table có các keys là ['table_content', 'page_numbers', 'source', 'table_idx']
        ## Trong đó table_content là markdown của bảng
        ## page_numbers là list các page number mà bảng xuất hiện
        ## source là tên của pdf
        ## table_idx là index của bảng trong pdf
        print(table['table_content'])
        print("-" * 100)

source c46a0e327c07d6d3a04b23f7de59b55d has 2 tables
----------------------------------------------------------------------------------------------------
# c46a0e327c07d6d3a04b23f7de59b55d_table_0
This is a cross-page table. It spans multiple pages. Page numbers: [3, 4, 5, 6]
| Year       | Title                                                               | Role [ citation needed ]           | Notes                                                                       |
|:-----------|:--------------------------------------------------------------------|:-----------------------------------|:----------------------------------------------------------------------------|
| 1979       | Buck Rogers in the 25th Century                                     | Colonel Wilma Deering              |                                                                             |
| 1979       | Winter Kills                                                        | Beautiful Woman Three              |  

In [ ]:

class QAPair(BaseModel):
    id: str
    source: str
    page_numbers: List[int]
    table_idx: int
    question: str
    answer: str
    context: str


from typing import List

class QAPairList(BaseModel):
    qa_pairs: List[QAPair]

In [ ]:


credentials_path = "C:/Users/Admin/Data/multimodal-rag-baseline/gdsc2025-74596a254ab4.json"

credentials = service_account.Credentials.from_service_account_file(credentials_path)


In [18]:
openai_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))
llm = ChatVertexAI(model="gemini-2.0-flash", temperature=0)
# llm.invoke("hello world")
structured_llm = llm.with_structured_output(QAPairList)


C:\Users\Admin\AppData\Local\Temp\ipykernel_24836\946967047.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  openai_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))


In [ ]:


gemini_prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Your task is to generate exactly 5 question-answer pairs based on the following table:

{table_content}

Please return exactly 5 QA pairs in this JSON format:
[
  {{
    "id": "qa_{counter}_001",
    "source": "{source}",
    "page_numbers": {page_numbers},
    "table_idx": {table_idx},
    "question": "Sample question",
    "answer": "Sample answer",
    "context": "{table_content}"
  }},
  ...
]

Make sure:
- Each question must be of different types and focus on specific data from the table.
- Include at least two questions that require **listing**, **comparing**, or **analyzing trends/events** (e.g., "List all movies released between 2000 and 2010", "What are the top 3 events in chronological order?", "Which country had the highest population growth?").
- Avoid simple lookup questions — aim for medium to high difficulty.
- Answers must come directly from the table.
- DO NOT include any extra text — just return the JSON array.
"""
)

tables_sources = json.load(open("results.json"))
qa_list = []
qa_global_counter = 1

for source in tables_sources:
    print(f"Processing source: {source}")
    for idx, table in enumerate(tables_sources[source]):
        print(f" - Table index {idx}...")

        table_content = table["table_content"]
        page_numbers = table["page_numbers"]
        table_idx = table["table_idx"]

        try:
            custom_prompt = gemini_prompt.template.replace("{counter}", str(qa_global_counter))
            formatted_prompt = custom_prompt.format(
                table_content=table_content,
                source=source,
                page_numbers=page_numbers,
                table_idx=table_idx
            )

            print(f"Formatted prompt: {formatted_prompt}")
            response = structured_llm.invoke(formatted_prompt)
            print(f"Response: {response}")

            if hasattr(response, "model_dump"):
                generated_qas = [item.model_dump() for item in response.qa_pairs]  # hoặc item.dict()
            elif hasattr(response, "__dict__"):
                generated_qas = [dict(item) for item in response]
            else:
                clean_text = re.sub(r'```json\n|\n```', '', str(response)).strip()
                generated_qas = json.loads(clean_text)

            qa_list.extend(generated_qas)

            qa_global_counter += 5

        except Exception as e:
            print(f"[ERROR] Failed to parse Gemini output: {e}")
            continue

with open("generated_qa_pairs.json", "w", encoding="utf-8") as f:
    json.dump(qa_list, f, indent=2, ensure_ascii=False)

Processing source: c46a0e327c07d6d3a04b23f7de59b55d
 - Table index 0...
Formatted prompt: 
You are a helpful assistant. Your task is to generate exactly 5 question-answer pairs based on the following table:

# c46a0e327c07d6d3a04b23f7de59b55d_table_0
This is a cross-page table. It spans multiple pages. Page numbers: [3, 4, 5, 6]
| Year       | Title                                                               | Role [ citation needed ]           | Notes                                                                       |
|:-----------|:--------------------------------------------------------------------|:-----------------------------------|:----------------------------------------------------------------------------|
| 1979       | Buck Rogers in the 25th Century                                     | Colonel Wilma Deering              |                                                                             |
| 1979       | Winter Kills                                          

In [27]:
from langchain.prompts import PromptTemplate

evaluation_prompt = PromptTemplate.from_template(
    """
You are an expert in evaluating the accuracy of answers to questions. 
Given the question, model-generated answer, and a golden (correct) answer, you should rate the model's answer on a scale from 0 to 1, where:

- 1.0 means the model answer is fully correct.
- 0.5 means partially correct or incomplete.
- 0.0 means incorrect or irrelevant.

Evaluate strictly based on factual correctness and alignment with the golden answer.

Question: {question}
Model Answer: {model_answer}
Golden Answer: {golden_answer}

Score:
"""
)

from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain


eval_chain = LLMChain(llm=openai_model, prompt=evaluation_prompt)

C:\Users\Admin\AppData\Local\Temp\ipykernel_24836\4182186215.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  eval_chain = LLMChain(llm=openai_model, prompt=evaluation_prompt)


In [29]:
import json

with open("generated_qa_pairs.json", "r", encoding="utf-8") as f:
    qa_list = json.load(f)

evaluation_results = []

for i, item in enumerate(qa_list):
    try:
        result = eval_chain.invoke({
            "question": item["question"],
            "model_answer": item["answer"],
            "golden_answer": item["answer"]  # giả lập golden answer
        })

        raw_score = result["text"].strip()
        score = float(raw_score) if raw_score.replace('.', '', 1).isdigit() else 0.0

        evaluation_results.append({
            "id": item["id"],
            "question": item["question"],
            "model_answer": item["answer"],
            "score": score
        })

        print(f"[QA {i+1}] ID: {item['id']} | Score: {score}")

    except Exception as e:
        print(f"[ERROR] Failed to evaluate QA {i+1}: {e}")
        evaluation_results.append({
            "id": item["id"],
            "question": item["question"],
            "model_answer": item["answer"],
            "score": 0.0
        })

[QA 1] ID: qa_1_001 | Score: 1.0
[QA 2] ID: qa_1_002 | Score: 1.0
[QA 3] ID: qa_1_003 | Score: 1.0
[QA 4] ID: qa_1_004 | Score: 1.0
[QA 5] ID: qa_1_005 | Score: 1.0
[QA 6] ID: qa_6_001 | Score: 1.0
[QA 7] ID: qa_6_002 | Score: 1.0
[QA 8] ID: qa_6_003 | Score: 1.0
[QA 9] ID: qa_6_004 | Score: 1.0
[QA 10] ID: qa_6_005 | Score: 1.0
[QA 11] ID: qa_11_001 | Score: 1.0
[QA 12] ID: qa_11_002 | Score: 1.0
[QA 13] ID: qa_11_003 | Score: 0.0
[QA 14] ID: qa_11_004 | Score: 1.0
[QA 15] ID: qa_11_005 | Score: 1.0
[QA 16] ID: qa_16_001 | Score: 1.0
[QA 17] ID: qa_16_002 | Score: 1.0
[QA 18] ID: qa_16_003 | Score: 1.0
[QA 19] ID: qa_16_004 | Score: 1.0
[QA 20] ID: qa_16_005 | Score: 1.0
[QA 21] ID: qa_21_001 | Score: 1.0
[QA 22] ID: qa_21_002 | Score: 1.0
[QA 23] ID: qa_21_003 | Score: 1.0
[QA 24] ID: qa_21_004 | Score: 1.0
[QA 25] ID: qa_21_005 | Score: 1.0
[QA 26] ID: qa_26_001 | Score: 1.0
[QA 27] ID: qa_26_002 | Score: 1.0
[QA 28] ID: qa_26_003 | Score: 1.0
[QA 29] ID: qa_26_004 | Score: 1.0
[QA 